In [ ]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from itertools import combinations
from capture import CAMERA_WALL_DISTANCE

In [ ]:
data_dir = Path("data")

devices = {}

for device_dir in data_dir.iterdir():
	if not device_dir.is_dir():
		continue
	print(device_dir.name)

	images = {}
	for image_dir in device_dir.glob("*.png"):
		images[image_dir.stem] = cv2.imread(str(image_dir))

	devices[device_dir.name] = images



In [ ]:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

def get_corners(image):
	""" Get the corners of the noise pattern board """
	
	corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(image, aruco_dict)
	inds = np.argsort(ids.flatten())
	corners_sorted = np.array(corners)[inds]
	if len(corners_sorted) != 4:
		raise Exception("Noise pattern board not found")

	center = np.array([m.mean(axis=1).flatten() for m in corners_sorted]).mean(axis=0)

	points = []
	for m in corners_sorted:
		i = np.linalg.norm(m - center, axis=-1).argmin()
		points.append(m[:, i])

	points = np.array(points).reshape(-1, 2)

	return points

def get_center(points):
	center = points.mean(axis=0)
	return center[0], center[1]

def get_rotation(points):
	center_bottom = (points[0] + points[1]) / 2
	center_top = (points[2] + points[3]) / 2
	d = center_bottom - center_top
	angle = np.arctan2(*d)
	return angle

def get_board_width(points):
	return np.linalg.norm(points[0] - points[1])
	

In [ ]:
center_bottom = np.array([-4, 600])
center_top = np.array([0, 0])
d = center_bottom - center_top
angle = np.arctan2(*d)
angle_deg = np.rad2deg(angle)
print(angle_deg)

In [ ]:
results = []

offsets = { # compensation for different camera positions (in m)
    "right": -0.0375,
    "rectified_right": -0.0375,
    "color": 0,
    "left": 0.0375,
    "rectified_left": 0.0375,
}

for mxid, images in devices.items():
    result = {"mxid": mxid}

    for name, img in images.items():
        points = get_corners(img)
        center_x, center_y = get_center(points)
        angle = get_rotation(points)
        board_width = get_board_width(points)
        px2m = 0.6 / board_width

        offset = offsets[name]

        result[f"{name}_roll_angle"] = np.rad2deg(angle)
        result[f"{name}_yaw_angle"] = np.rad2deg(np.arctan2(offset + px2m * (img.shape[1]/2 - center_x), CAMERA_WALL_DISTANCE))
        result[f"{name}_pitch_angle"] = np.rad2deg(np.arctan2(px2m * (img.shape[0]/2 - center_y), CAMERA_WALL_DISTANCE))


    for cam_a, cam_b in combinations(["left", "right", "color"], 2):
        result[f"{cam_a}_{cam_b}_roll_diff"] = np.abs(result[f"{cam_a}_roll_angle"] - result[f"{cam_b}_roll_angle"])
        result[f"{cam_a}_{cam_b}_yaw_diff"] = np.abs(result[f"{cam_a}_yaw_angle"] - result[f"{cam_b}_yaw_angle"])
        result[f"{cam_a}_{cam_b}_pitch_diff"] = np.abs(result[f"{cam_a}_pitch_angle"] - result[f"{cam_b}_pitch_angle"])

    for cam_a, cam_b in combinations(["rectified_left", "rectified_right"], 2):
        result[f"{cam_a}_{cam_b}_roll_diff"] = np.abs(result[f"{cam_a}_roll_angle"] - result[f"{cam_b}_roll_angle"])
        result[f"{cam_a}_{cam_b}_yaw_diff"] = np.abs(result[f"{cam_a}_yaw_angle"] - result[f"{cam_b}_yaw_angle"])
        result[f"{cam_a}_{cam_b}_pitch_diff"] = np.abs(result[f"{cam_a}_pitch_angle"] - result[f"{cam_b}_pitch_angle"])

    results.append(result)

results_df = pd.DataFrame(results)
results_df.transpose()

In [ ]:
results_df[["right_color_roll_diff", "right_color_yaw_diff", "right_color_pitch_diff"]].mean()

In [ ]:
results_df.std(numeric_only=True)

In [ ]:
np.mean(np.abs(results_df["color_roll_angle"].to_numpy()))

In [ ]:
mount_roll_angle = np.mean([
	np.mean(results_df["color_roll_angle"].to_numpy()),
	np.mean(results_df["left_roll_angle"].to_numpy()),
	np.mean(results_df["right_roll_angle"].to_numpy())
])

mount_yaw_angle = np.mean([
	np.mean(results_df["color_yaw_angle"].to_numpy()),
	np.mean(results_df["left_yaw_angle"].to_numpy()),
	np.mean(results_df["right_yaw_angle"].to_numpy())
])

mount_pitch_angle = np.mean([
	np.mean(results_df["color_pitch_angle"].to_numpy()),
	np.mean(results_df["left_pitch_angle"].to_numpy()),
	np.mean(results_df["right_pitch_angle"].to_numpy())
])

print(f"mount_roll_angle: {mount_roll_angle}")
print(f"mount_yaw_angle: {mount_yaw_angle}")
print(f"mount_pitch_angle: {mount_pitch_angle}")

results_corrected = results_df.copy()
results_corrected["color_roll_angle"] -= mount_roll_angle
results_corrected["left_roll_angle"] -= mount_roll_angle
results_corrected["right_roll_angle"] -= mount_roll_angle
results_corrected["rectified_right_roll_angle"] -= mount_roll_angle
results_corrected["rectified_left_roll_angle"] -= mount_roll_angle

results_corrected["color_yaw_angle"] -= mount_yaw_angle
results_corrected["left_yaw_angle"] -= mount_yaw_angle
results_corrected["right_yaw_angle"] -= mount_yaw_angle
results_corrected["rectified_right_yaw_angle"] -= mount_yaw_angle
results_corrected["rectified_left_yaw_angle"] -= mount_yaw_angle

results_corrected["color_pitch_angle"] -= mount_pitch_angle
results_corrected["left_pitch_angle"] -= mount_pitch_angle
results_corrected["right_pitch_angle"] -= mount_pitch_angle
results_corrected["rectified_right_pitch_angle"] -= mount_pitch_angle
results_corrected["rectified_left_pitch_angle"] -= mount_pitch_angle



In [ ]:
results_corrected["color_roll_angle_ok"] = np.abs(results_corrected["color_roll_angle"]) < 0.5
results_corrected["color_yaw_angle_ok"] = np.abs(results_corrected["color_yaw_angle"]) < 1.0
results_corrected["color_pitch_angle_ok"] = np.abs(results_corrected["color_pitch_angle"]) < 1.2

results_corrected["left_roll_angle_ok"] = np.abs(results_corrected["left_roll_angle"]) < 0.5
results_corrected["left_yaw_angle_ok"] = np.abs(results_corrected["left_yaw_angle"]) < 2.0
results_corrected["left_pitch_angle_ok"] = np.abs(results_corrected["left_pitch_angle"]) < 1.5

results_corrected["rectified_right_roll_angle_ok"] = np.abs(results_corrected["rectified_right_roll_angle"]) < 0.5
results_corrected["rectified_right_yaw_angle_ok"] = np.abs(results_corrected["rectified_right_yaw_angle"]) < 2.0
results_corrected["rectified_right_pitch_angle_ok"] = np.abs(results_corrected["rectified_right_pitch_angle"]) < 1.5

results_corrected["rectified_left_roll_angle_ok"] = np.abs(results_corrected["rectified_left_roll_angle"]) < 0.5
results_corrected["rectified_left_yaw_angle_ok"] = np.abs(results_corrected["rectified_left_yaw_angle"]) < 2.0
results_corrected["rectified_left_pitch_angle_ok"] = np.abs(results_corrected["rectified_left_pitch_angle"]) < 1.5

results_corrected["right_roll_angle_ok"] = np.abs(results_corrected["right_roll_angle"]) < 0.5
results_corrected["right_yaw_angle_ok"] = np.abs(results_corrected["right_yaw_angle"]) < 2.0
results_corrected["right_pitch_angle_ok"] = np.abs(results_corrected["right_pitch_angle"]) < 1.5


# results_corrected["left_right_roll_diff_ok"] = results_corrected["left_right_roll_diff"] < 0.3
# results_corrected["left_right_yaw_diff_ok"] = results_corrected["left_right_yaw_diff"] < 0.8
# results_corrected["left_right_pitch_diff_ok"] = results_corrected["left_right_pitch_diff"] < 0.15



In [ ]:
results_corrected.transpose().to_excel("results.xlsx")

In [ ]:
list(combinations([1,2,3], r=2))